# About

This notebook downloads canopy height data (10m resolution) over Santa Barabara County from the [California Forest Observatory (CFO) API](https://github.com/forestobservatory/cfo-api). To do this we follow the 'Custom Data Clipping' section in [this CFO API tutorial](https://github.com/forestobservatory/cfo-api/blob/master/demos/api-introduction.ipynb). A CFO account is needed to run the notebook. 

[Source](https://databasin.org/datasets/1cdc07adea7d4dee9cc1f07ab44cfef3/) of the Santa Barbara County shapefile. This was previously converted to a geojson file using geopandas.

**NOTEBOOK VARIABLES**: 
- `years` (list of int): years for which we will access the CFO California canopy height rasters. Must be a subset of [2016, 2018, 2020] since CFO's canopy height data is not available for years before 2016. 

**OUTPUT**:
-  canopy height rasters for Santa Barbara County






# Install California Forest Observatory API package if not available

In [1]:
#pip install cfo

  Using cached cfo-1.0.1-py3-none-any.whl (28 kB)
  Using cached retrying-1.3.3-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

import cfo
from osgeo import gdal

# Specify years for canopy height data

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

# years should be a subset of [2016, 2018, 2020]
years = [2016, 2018, 2020]

# ***************************************************
# ***************************************************

# Authenticate 

In [4]:
forest_api = cfo.api()
forest_api.authenticate()  #galaz-garcia@nceas.ucsb.edugalaz-garcia@nceas.ucsb.edu

CFO E-mail:  galaz-garcia@nceas.ucsb.edu
CFO Password:  ···········


200

# Download canopy height

In [5]:
# load SB shape
vector = os.path.join(os.getcwd(),'SantaBarbaraCounty_geojson','SantaBarbaraCounty.geojson')

for year in years:
    # get the 10m canopy height asset id for the given year
    search = forest_api.search(geography='California', metric='CanopyHeight', year=year, resolution=10)
    itemid = search[0]

    # and get the gdal file path reference
    input_file = forest_api.fetch(itemid, gdal=True)

    # set the output raster file
    fp = os.path.join(os.getcwd(),'SantaBarbaraCounty_CanopyHeight')
    if os.path.exists(fp) == False:
        os.mkdir(fp)
    output_file = os.path.join(fp, 'SantaBarbaraCounty_CanopyHeight_'+str(year)+'.tif')

    # set the gdalwarp options
    options = gdal.WarpOptions(
        creationOptions = ["COMPRESS=DEFLATE", "TILED=YES", "BIGTIFF=YES", "NUM_THREADS=ALL_CPUS"],
        cutlineDSName = vector,
        cropToCutline = True
    )

    # and run the command
    warp = gdal.Warp(output_file, input_file, options=options)
    warp.FlushCache()